In [1]:
import pandas as pd
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install ragas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access the variables
openai_api_key = os.getenv('OPENAI_API_KEY')
infer_endpoint = os.getenv("INFER_ENDPOINT")
model_name = os.getenv("MODEL_NAME")
api_key = os.getenv("LLM_API_KEY")

data_dir = os.getenv('DATA_DIR')
output_dir = os.getenv('OUTPUT_DIR')


In [15]:
from datasets import Dataset, load_dataset
import pandas as pd

testset_json =f"{output_dir}/testset.json"
testset = Dataset.from_pandas(pd.read_json(testset_json))
testset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', '__index_level_0__'],
    num_rows: 20
})

In [16]:
from langchain_community.llms import VLLMOpenAI
from langchain.prompts import PromptTemplate

# LLM definition
llm = VLLMOpenAI(           # we are using the vLLM OpenAI-compatible API client. But the Model is running on OpenShift, not OpenAI.
    openai_api_key=api_key,   # and that is why we don't need an OpenAI key for this.
    openai_api_base= f"{infer_endpoint}/v1",
    model_name=f"{model_name}",
    temperature=0.00,
    max_tokens=2048
)

In [17]:
TEXT_QA_TEMPLATE = (
    "<|system|>\n"
    "You are, Red Hat Instruct Model based on Granite 7B, an AI language model developed by Red Hat and IBM Research, based on the Granite-7b-base language model. My primary function is to be a chat assistant.\n"
    "<|user|>\n"
    "{context_str}\n"
    "Given the above context information, answer the query.\n"
    "{query_str}\n"
    "<|assistant|>\n"
)


PROMPT = PromptTemplate(input_variables=["query_str", "context_str"], template=TEXT_QA_TEMPLATE)


In [18]:
from langchain.chains import LLMChain

def generate_answers(testset, prompt, llm):
    dataset_df = testset.to_pandas()
    dataset_df["answer"] = None
    
    for index, row in dataset_df.iterrows():
        conversation = LLMChain(
            llm=llm,
            prompt=prompt,
            verbose=False
        )
        query_str = row["question"]
        context_str = "\n\n".join(row["contexts"])
        answer = conversation.predict(query_str=query_str, context_str=context_str)
        print(query_str[:50], answer[:100])
        dataset_df.loc[index, "answer"] = answer
        
    return Dataset.from_pandas(dataset_df)

In [19]:
answers_ds = generate_answers(testset, PROMPT, llm)
answers_ds.to_pandas()

Samsung SDS is working to upgrade their DevSecOps 
One of the factors that can be assessed in the hum
The Certification Support System at Samsung SDS is
The Board of Directors (BOD) in Samsung SDS is res
The Stepping Stone of Hope Center is a program ope
The purpose of the training provided for independe
The RE:CYCLE campaign conducted by Samsung SDS aim
Samsung SDS has a system in place to respond promp
Samsung SDS has registered a total of 4,758 patent
Samsung SDS requires suppliers to comply with all 
Samsung SDS protects customer privacy and personal
Samsung SDS supports suppliers in preventing accid
The CFO plays a significant role in the ESG Commit
The Software Promotion Act grants software process
Samsung SDS's Safety & Health policy aims to prote
The company's Information Security Policies apply 
Samsung SDS has implemented several waste reductio
The Certification Support System at Samsung SDS is
Samsung SDS offers the Object Oriented methodology
Samsung SDS takes several actio

,question,contexts,ground_truth,evolution_type,metadata,episode_done,__index_level_0__,answer
0,How does Samsung SDS aim to upgrade their DevS...,[\n\nSOCIAL\n\nGOVERNANCE\n\n## **Sustainable ...,Samsung SDS aims to upgrade their DevSecOps le...,simple,[{'source': 'output/Samsung/md/document_257.md'}],True,0,Samsung SDS is working to upgrade their DevSec...
1,What is one of the factors that can be assesse...,[|-------------------------------------------|...,The work environment is one of the factors tha...,simple,[{'source': 'output/Samsung/md/document_787.md'}],True,1,One of the factors that can be assessed in the...
2,What does the Certification Support System at ...,[## **Regional Specialist Program**\n\nSamsung...,The Certification Support System at Samsung SD...,simple,[{'source': 'output/Samsung/md/document_248.md'}],True,2,The Certification Support System at Samsung SD...
3,What is the role of the Board of Directors in ...,[Samsung SDS designates a supporting departmen...,The role of the Board of Directors in Samsung ...,simple,[{'source': 'output/Samsung/md/document_299.md'}],True,3,The Board of Directors (BOD) in Samsung SDS is...
4,What is the purpose or function of the Steppin...,"[Self-reliance practice 2,600 people\n\nSteppi...",The purpose or function of the Stepping Stone ...,simple,[{'source': 'output/Samsung/md/document_463.md'}],True,4,The Stepping Stone of Hope Center is a program...
5,What is the purpose of the training provided f...,[Samsung SDS designates a supporting departmen...,Samsung SDS designates a supporting department...,simple,[{'source': 'output/Samsung/md/document_482.md'}],True,5,The purpose of the training provided for indep...
6,What is the purpose of the RE:CYCLE campaign c...,"[In line with the ESG business trend, many com...",The purpose of the RE:CYCLE campaign conducted...,simple,[{'source': 'output/Samsung/md/document_52.md'}],True,6,The RE:CYCLE campaign conducted by Samsung SDS...
7,What is the process for responding to personal...,[## **Process for Responding to Personal Data ...,In accordance with the Personal Information Pr...,simple,[{'source': 'output/Samsung/md/document_839.md'}],True,7,Samsung SDS has a system in place to respond p...
8,How many registered patents does Samsung SDS h...,"[As of the end of 2023, Samsung SDS operates o...","Total 4,758 persons (cumulative)",simple,[{'source': 'output/Samsung/md/document_189.md'}],True,8,"Samsung SDS has registered a total of 4,758 pa..."
9,What standards and guidelines does Samsung SDS...,[Suppliers must comply with all applicable law...,Samsung SDS requires suppliers to comply with ...,simple,[{'source': 'output/Samsung/md/document_674.md'}],True,9,Samsung SDS requires suppliers to comply with ...


In [9]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
)

metrics = [
    faithfulness,
    answer_relevancy,
    answer_correctness,
]

In [20]:
answers_ds

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', '__index_level_0__', 'answer'],
    num_rows: 20
})

In [21]:
eval_result = evaluate(
    answers_ds,
    metrics=metrics,
)
eval_result

Evaluating: 100%|██████████| 60/60 [01:23<00:00,  1.40s/it]


{'faithfulness': 0.6983, 'answer_relevancy': 0.9666, 'answer_correctness': 0.7650}

In [22]:
eval_result.to_pandas()


,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,answer_correctness
0,How does Samsung SDS aim to upgrade their DevS...,[\n\nSOCIAL\n\nGOVERNANCE\n\n## **Sustainable ...,Samsung SDS is working to upgrade their DevSec...,Samsung SDS aims to upgrade their DevSecOps le...,0.600000,0.986176,0.996539
1,What is one of the factors that can be assesse...,[|-------------------------------------------|...,One of the factors that can be assessed in the...,The work environment is one of the factors tha...,0.600000,0.983048,0.487512
2,What does the Certification Support System at ...,[## **Regional Specialist Program**\n\nSamsung...,The Certification Support System at Samsung SD...,The Certification Support System at Samsung SD...,1.000000,0.967314,0.694957
3,What is the role of the Board of Directors in ...,[Samsung SDS designates a supporting departmen...,The Board of Directors (BOD) in Samsung SDS is...,The role of the Board of Directors in Samsung ...,0.000000,1.000000,0.653886
4,What is the purpose or function of the Steppin...,"[Self-reliance practice 2,600 people\n\nSteppi...",The Stepping Stone of Hope Center is a program...,The purpose or function of the Stepping Stone ...,0.076923,0.994247,0.207982
5,What is the purpose of the training provided f...,[Samsung SDS designates a supporting departmen...,The purpose of the training provided for indep...,Samsung SDS designates a supporting department...,0.750000,1.000000,0.500891
6,What is the purpose of the RE:CYCLE campaign c...,"[In line with the ESG business trend, many com...",The RE:CYCLE campaign conducted by Samsung SDS...,The purpose of the RE:CYCLE campaign conducted...,1.000000,1.000000,0.952621
7,What is the process for responding to personal...,[## **Process for Responding to Personal Data ...,Samsung SDS has a system in place to respond p...,In accordance with the Personal Information Pr...,1.000000,0.957219,0.578888
8,How many registered patents does Samsung SDS h...,"[As of the end of 2023, Samsung SDS operates o...","Samsung SDS has registered a total of 4,758 pa...","Total 4,758 persons (cumulative)",0.000000,0.993712,0.698845
9,What standards and guidelines does Samsung SDS...,[Suppliers must comply with all applicable law...,Samsung SDS requires suppliers to comply with ...,Samsung SDS requires suppliers to comply with ...,1.000000,0.943365,0.998080


In [23]:
eval_result.to_pandas().to_csv(f"{output_dir}/ragas_eval_result.csv", index=False, header=True)